In [ ]:
!pip install git+https://github.com/forrestbao/pyeeg.git
import numpy as np
import pyeeg as pe
import pickle as pickle
import pandas as pd
import math
from sklearn import svm
from sklearn.preprocessing import normalize

import os
import time

  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-mrf33r_o
  Running command git clone --filter=blob:none --quiet https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-mrf33r_o
  Resolved https://github.com/forrestbao/pyeeg.git to commit a6c18bb093e4748f9d9c208535a6ae024a0802b8
  Preparing metadata (setup.py) ... done
  Created wheel for pyeeg: filename=pyeeg-0.4.4-py2.py3-none-any.whl size=28111 sha256=d79f77fac100b74f550827746ecff58bbd23099c89c571885cdd865385328dc2
  Stored in directory: /tmp/pip-ephem-wheel-cache-at9__a90/wheels/a8/c4/1a/cee09dcc12a11620066d35ace42e3c1e3bfbcc1db3a0ce7788
Successfully built pyeeg


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
band = [4,8,12,16,25,45] #5 bands
#channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
channel = [2,7,13,24,29] #5 Channels chosen to fit Emotiv Ensight
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = [str(i).zfill(2) for i in range(1, 33)]
#List of subjects

In [ ]:
import numpy as np
import pickle
import os

def FFT_Processing(sub, channel, band, window_size, step_size, sample_rate):
    meta = []
    with open(os.path.join('/content/drive/MyDrive/data_preprocessed_python', 's' + sub + '.dat'), 'rb') as file:
        subject = pickle.load(file, encoding='latin1') # resolve the python 2 data problem by encoding: latin1

        for i in range(0, 40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0

            while start + window_size < data.shape[1]:
                meta_data = [] # meta vector for analysis
                for j in channel:
                    X = data[j-1][start: start + window_size] # Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = pe.bin_power(X, band, sample_rate) # FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    meta_data.extend(Y[0])

                meta_array = [np.array(meta_data), labels] # Use list to contain meta_data and labels
                meta.append(meta_array) # Append the list to meta
                start += step_size

    # Convert meta to an array if needed right before saving
    np.save(os.path.join('/content/drive/MyDrive/data_preprocessed_python/Deap5_FFT', 's' + sub), np.array(meta, dtype=object), allow_pickle=True)

In [ ]:
for subjects in subjectList:
    FFT_Processing (subjects, channel, band, window_size, step_size, sample_rate)


In [ ]:

data_training = []
label_training = []
data_testing = []
label_testing = []

for subjects in subjectList:

    with open('/content/drive/MyDrive/data_preprocessed_python/Deap5_FFT/s' + subjects + '.npy', 'rb') as file:
      sub = np.load(file,allow_pickle=True)
      for i in range (0,sub.shape[0]):
        if i % 5 == 0:
          data_testing.append(sub[i][0])
          label_testing.append(sub[i][1])
        else:
          data_training.append(sub[i][0])
          label_training.append(sub[i][1])

np.save('/content/drive/MyDrive/data_preprocessed_python/Deap5_FFT/data_training', np.array(data_training), allow_pickle=True, fix_imports=True)
np.save('/content/drive/MyDrive/data_preprocessed_python/Deap5_FFT/label_training', np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)

np.save('/content/drive/MyDrive/data_preprocessed_python/Deap5_FFT/data_testing', np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save('/content/drive/MyDrive/data_preprocessed_python/Deap5_FFT/label_testing', np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)